# TO EXTRACT NAME


In [24]:
import cv2
import pytesseract
from rapidfuzz import process, fuzz

# 1) Configure Tesseract executable path if needed (comment out if already in PATH).

pytesseract.pytesseract.tesseract_cmd = '/opt/anaconda3/envs/digithon/bin/tesseract'

# 2) Suppose you have a known list of medicine names in your dataset or domain.
#    Feel free to add/remove names as needed:
df1 = pd.read_csv('A_Z_medicines_dataset_of_India.csv')
KNOWN_MEDICINE_NAMES = df1['name'].dropna().tolist()

def extract_medicine_name(image_path):
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"[Error] Could not read image: {image_path}")
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Optional: Preprocessing (e.g., thresholding) to improve OCR accuracy
    # You can experiment with different thresholds or filters
    # to see what works best for your images.
    # For example:
    # threshold_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    threshold_img = gray  # using plain grayscale for simplicity

    # 3) Extract text via Tesseract
    extracted_text = pytesseract.image_to_string(threshold_img)

    # Clean up text
    cleaned_text = " ".join(extracted_text.strip().split())
    print("Raw OCR text:", cleaned_text)
    
    # 4) Use fuzzy matching to find the best match from KNOWN_MEDICINE_NAMES
    best_match, score, _ = process.extractOne(
        cleaned_text,
        KNOWN_MEDICINE_NAMES,
        scorer=fuzz.token_set_ratio
    )

    # 5) Set a threshold to accept the match
    threshold = 20  # adjust as needed
    if score >= threshold:
        return best_match
    else:
        # If no match is above threshold, return the raw extracted text
        # or None, depending on your preference.
        return None

if __name__ == "__main__":
    # Example usage with your images
    image_paths = [
        "1.jpeg"      # Replace with the actual path to the Dolo-650 image
    ]
    
    for path in image_paths:
        medicine_name = extract_medicine_name(path)
        if medicine_name:
            print(f"Extracted Medicine Name: {medicine_name}")
        else:
            print("No medicine name confidently recognized.")
    



Raw OCR text: Paracetamol. “Tablets IP 500 mg eee Paracemeter-500 TABLETS
Extracted Medicine Name: Jackson Paracetamol 500 Tablet


# This code returns the best match


In [ ]:

import cv2
import pytesseract
import pandas as pd
from rapidfuzz import process, fuzz
from tabulate import tabulate

# Configure Tesseract executable path if needed
pytesseract.pytesseract.tesseract_cmd = '/opt/anaconda3/envs/digithon/bin/tesseract'

# Load your dataset and extract the list of known medicine names
df1 = pd.read_csv('A_Z_medicines_dataset_of_India.csv')
KNOWN_MEDICINE_NAMES = df1['name'].dropna().tolist()

def extract_medicine_name(image_path):
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"[Error] Could not read image: {image_path}")
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Preprocessing (e.g., thresholding) can be added here if needed
    threshold_img = gray  # using plain grayscale for simplicity

    # Extract text via Tesseract
    extracted_text = pytesseract.image_to_string(threshold_img)
    cleaned_text = " ".join(extracted_text.strip().split())
    print("Raw OCR text:", cleaned_text)
    
    # Use fuzzy matching to find the best match from KNOWN_MEDICINE_NAMES
    best_match, score, _ = process.extractOne(
        cleaned_text,
        KNOWN_MEDICINE_NAMES,
        scorer=fuzz.token_set_ratio
    )

    # Set a threshold to accept the match
    threshold =55  # adjust as needed
    if score >= threshold:
        return best_match
    else:
        return None

if __name__ == "__main__":
    image_paths = ["2.jpg"]  # Replace with your image paths
    
    for path in image_paths:
        medicine_name = extract_medicine_name(path)
        if medicine_name:
            print(f"\nExtracted Medicine Name: {medicine_name}\n")
            # Filter and show all values (columns) for the matched medicine from the dataset
            matched_rows = df1[df1['name'] == medicine_name]
            if not matched_rows.empty:
                # Print in a nicely formatted table using tabulate
                print("Full details of the medicine:")
                print(tabulate(matched_rows, headers='keys', tablefmt='psql', showindex=False))
            else:
                print("Matched name not found in the dataset.")
        else:
            print("No medicine name confidently recognized.")


Raw OCR text: 010-8 Dake Pose =pn® "Dolo-650° ates, uncoated tablet contains: stamol IP. 650 mig. age: Asiected by the P In alt & dorkplace, 55 fc temperature not NiSotsinga ie

Extracted Medicine Name: P 650 Tablet

Full details of the medicine:
+--------+--------------+------------+-------------------+---------------------------+-----------+---------------------+----------------------+----------------------+
|     id | name         |   price(₹) | Is_discontinued   | manufacturer_name         | type      | pack_size_label     | short_composition1   |   short_composition2 |
|--------+--------------+------------+-------------------+---------------------------+-----------+---------------------+----------------------+----------------------|
| 173668 | P 650 Tablet |      22.85 | False             | Apex Laboratories Pvt Ltd | allopathy | strip of 10 tablets | Paracetamol (650mg)  |                  nan |
+--------+--------------+------------+-------------------+---------------------------

# This code returns all the matches in a tabulate formate

In [9]:
import cv2
import pytesseract
import pandas as pd
from rapidfuzz import process, fuzz
from tabulate import tabulate

# Configure Tesseract executable path if needed
pytesseract.pytesseract.tesseract_cmd = '/opt/anaconda3/envs/digithon/bin/tesseract'

# Load your dataset and extract the list of known medicine names
df1 = pd.read_csv('A_Z_medicines_dataset_of_India.csv')
KNOWN_MEDICINE_NAMES = df1['name'].dropna().tolist()

def extract_medicine_names(image_path, match_threshold=20, limit=10):
    """
    Extract text from the image and find all matching medicine names
    from KNOWN_MEDICINE_NAMES that have a score above match_threshold.
    """
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"[Error] Could not read image: {image_path}")
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # (Optional) Additional preprocessing can be applied here if needed.
    threshold_img = gray  # using plain grayscale for simplicity

    # Extract text via Tesseract
    extracted_text = pytesseract.image_to_string(threshold_img)
    cleaned_text = " ".join(extracted_text.strip().split())
    print("Raw OCR text:", cleaned_text)
    
    # Use fuzzy matching to find potential matches from KNOWN_MEDICINE_NAMES
    matches = process.extract(
        cleaned_text,
        KNOWN_MEDICINE_NAMES,
        scorer=fuzz.token_set_ratio,
        limit=limit
    )
    
    # Filter matches that meet the threshold
    valid_matches = [(name, score) for name, score, _ in matches if score >= match_threshold]
    return valid_matches

if __name__ == "__main__":
    image_paths = ["1.jpeg"]  # Replace with your image paths
    
    for path in image_paths:
        matches = extract_medicine_names(path, match_threshold=20, limit=10)
        if matches:
            print("\nMatched Medicines:")
            for medicine_name, score in matches:
                print(f"\nMedicine: {medicine_name} (Score: {score})")
                # Get full details from the dataset for each medicine match
                matched_rows = df1[df1['name'] == medicine_name]
                if not matched_rows.empty:
                    print(tabulate(matched_rows, headers='keys', tablefmt='psql', showindex=False))
                else:
                    print("No details found in the dataset.")
        else:
            print("No medicine names confidently recognized.")


Raw OCR text: Paracetamol. “Tablets IP 500 mg eee Paracemeter-500 TABLETS

Matched Medicines:

Medicine: Jackson Paracetamol 500 Tablet (Score: 56.17977528089887)
+--------+--------------------------------+------------+-------------------+------------------------------+-----------+---------------------+----------------------+----------------------+
|     id | name                           |   price(₹) | Is_discontinued   | manufacturer_name            | type      | pack_size_label     | short_composition1   |   short_composition2 |
|--------+--------------------------------+------------+-------------------+------------------------------+-----------+---------------------+----------------------+----------------------|
| 113932 | Jackson Paracetamol 500 Tablet |         16 | False             | Jackson Laboratories Pvt Ltd | allopathy | strip of 10 tablets | Paracetamol (500mg)  |                  nan |
+--------+--------------------------------+------------+-------------------+---------

# This code too return all in the tabulate format but also sorts it by price

In [14]:
import cv2
import pytesseract
import pandas as pd
from rapidfuzz import process, fuzz
from tabulate import tabulate

# Configure Tesseract executable path if needed
pytesseract.pytesseract.tesseract_cmd = '/opt/anaconda3/envs/digithon/bin/tesseract'

# Load your dataset and extract the list of known medicine names
df1 = pd.read_csv('A_Z_medicines_dataset_of_India.csv')
KNOWN_MEDICINE_NAMES = df1['name'].dropna().tolist()

def extract_medicine_names(image_path, match_threshold=20, limit=10):
    """
    Extract text from the image and find all matching medicine names
    from KNOWN_MEDICINE_NAMES that have a score above match_threshold.
    """
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"[Error] Could not read image: {image_path}")
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # (Optional) Additional preprocessing can be applied here if needed.
    threshold_img = gray  # using plain grayscale for simplicity

    # Extract text via Tesseract
    extracted_text = pytesseract.image_to_string(threshold_img)
    cleaned_text = " ".join(extracted_text.strip().split())
    print("Raw OCR text:", cleaned_text)
    
    # Use fuzzy matching to find potential matches from KNOWN_MEDICINE_NAMES
    matches = process.extract(
        cleaned_text,
        KNOWN_MEDICINE_NAMES,
        scorer=fuzz.token_set_ratio,
        limit=limit
    )
    
    # Filter matches that meet the threshold
    valid_matches = [(name, score) for name, score, _ in matches if score >= match_threshold]
    return valid_matches

if __name__ == "__main__":
    image_paths = ["1.jpeg"]  # Replace with your image paths
    all_matched_names = []  # To store names from all images (if more than one)

    for path in image_paths:
        matches = extract_medicine_names(path, match_threshold=20, limit=10)
        if matches:
            print("\nMatched Medicines:")
            for medicine_name, score in matches:
                print(f"\nMedicine: {medicine_name} (Score: {score})")
                # Get full details from the dataset for each medicine match
                matched_rows = df1[df1['name'] == medicine_name]
                if not matched_rows.empty:
                    print(tabulate(matched_rows, headers='keys', tablefmt='psql', showindex=False))
                    all_matched_names.append(medicine_name)
                else:
                    print("No details found in the dataset.")
        else:
            print("No medicine names confidently recognized.")

    # If we found any matches, combine all rows and sort them by price in ascending order
    if all_matched_names:
        combined_df = df1[df1['name'].isin(all_matched_names)]
        sorted_df = combined_df.sort_values(by='price(₹)', ascending=True)
        print("\nAll matched medicines sorted by price (₹) in ascending order:")
        print(tabulate(sorted_df, headers='keys', tablefmt='psql', showindex=False))


Raw OCR text: Paracetamol. “Tablets IP 500 mg eee Paracemeter-500 TABLETS

Matched Medicines:

Medicine: Jackson Paracetamol 500 Tablet (Score: 56.17977528089887)
+--------+--------------------------------+------------+-------------------+------------------------------+-----------+---------------------+----------------------+----------------------+
|     id | name                           |   price(₹) | Is_discontinued   | manufacturer_name            | type      | pack_size_label     | short_composition1   |   short_composition2 |
|--------+--------------------------------+------------+-------------------+------------------------------+-----------+---------------------+----------------------+----------------------|
| 113932 | Jackson Paracetamol 500 Tablet |         16 | False             | Jackson Laboratories Pvt Ltd | allopathy | strip of 10 tablets | Paracetamol (500mg)  |                  nan |
+--------+--------------------------------+------------+-------------------+---------

In [17]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.
